In [1]:
import io
import pandas as pd
import numpy as np
import scipy as sci

from sklearn.preprocessing import MinMaxScaler # found in the scikit-learn package
from numpy import loadtxt
#from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
#from keras.optimizers import SGD
#from keras.optimizers import Adam

import tensorflow as tf
from tensorflow.keras.models import Sequential
##from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
# The path where the CSV file is located
# Use your own path
path = "data-ts.csv"

# Read the CSV file and store it as a pandas DataFrame
df = pd.read_csv(path)

df.index = df.index + 1 #Initialize the index at 1

# Lag the data frame
data = pd.concat([df.shift(1), df.shift(2), df.shift(3), df], axis=1)
data.columns = ['x1','x2', 'x3', 'y']
data = data.iloc[4:,]
data.head()

FileNotFoundError: ignored

In [ ]:
# Pick 80% of the data but keeps the original sequence (R)
train = data[:int(data.shape[0]*0.8)]
test = data[int(data.shape[0]*0.8):]

# Clean data in case of missing data
data = data.dropna()
train = train.dropna()
test = test.dropna()

In [ ]:
# Calculate parameter for scaling back the outcome variable, both for the prediction and the actual values
ytrain_min = float(train.min()['y'])
ytrain_max = float(train.max()['y'])
ytrain_range = ytrain_max - ytrain_min

ytest_min = float(test.min()['y'])
ytest_max = float(test.max()['y'])
ytest_range = ytest_max - ytest_min

In [ ]:
# Normalize the data frame
def scale(data):
    scaled_data = data.copy()
    for column in data.columns: 
      scaled_data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())
    return scaled_data

In [ ]:
# Scale back the data frame
def scale_back(data, data_min, data_range):
    data_ = data.copy()
    data_.head()
    for column in data_.columns: 
        data_[column] = (data_range) * data_[column] + data_min
    return data_

In [ ]:
# Normalize the train data in the range of 0-1
train_ = scale(train)

In [ ]:
# Normalize the test data in the range of 0-1
test_ = scale(test)
test_


In [ ]:
# Separate the input (x) and output (y)
n = train_.columns                                    #column labels
xtrain = train_.filter(items = ['x1','x2','x3'])  #input of the training set
ytrain = train_.filter(items = ['y'])                    #output of the training set

xtest = test_.filter(items = ['x1','x2','x3'])  #input of the test set
ytest = test_.filter(items = ['y'])                    #output of the test set

ytest.head()


In [ ]:
# Train a neural network model
# https://www.tensorflow.org/api_docs/python/tf/keras/activations
model = Sequential()

# First add a layer with 3 neurons for 3 input variables (3 lags)
# and the RELU activation function
model.add(Dense(3, input_dim=3, activation='relu'))

# Then add a hidden layer with n neurons
# and the sigmoid activation function
model.add(Dense(3, activation='sigmoid'))

# Then add the output layer with 1 neuron
# and a linear activation function
model.add(Dense(1))

#model.build((None, 3))
model.summary()
print(model.count_params())

opt = tf.optimizers.Adam(learning_rate=0.0005)
print("")
print("Generating model ...")
# fix random seed for reproducibility
seed = 1
xtrain_for_parameter_fitting, x_validation, ytrain_for_parameter_fitting, y_validation = train_test_split(xtrain, ytrain, test_size=0.3, random_state=seed)

model.compile(loss='mse', optimizer=opt, metrics=['mean_squared_error'])
#history = model.fit(x=xtrain,y=ytrain, epochs=200, validation_split=0.3, verbose=0)
history = model.fit(x=xtrain_for_parameter_fitting,y=ytrain_for_parameter_fitting, epochs=200, validation_data=(x_validation,y_validation), verbose=0)


print("Model generation completed")


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

print("Final loss value in training set: ", loss[-1])
print("Final loss value in validation set: ", val_loss[-1])

# Plot learning curve
plt.plot(history.history['loss'], label='Loss (training data)')
plt.plot(history.history['val_loss'], label='Loss (validation data)')
plt.title('Learning curve')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
# Use the model to calculate predictions for the train set
y_pred = model.predict(xtrain_for_parameter_fitting)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns =['y'] 


In [ ]:
# Scale back the predictions and original values
y_pred_rescaled = scale_back(pd.DataFrame(y_pred), ytrain_min, ytrain_range)
y_pred_rescaled.columns =['y'] 
y_train_rescaled = scale_back(pd.DataFrame(ytrain_for_parameter_fitting), ytrain_min, ytrain_range)
y_train_rescaled.head()

In [ ]:
# r-squared
r2_squared = r2_score(y_train_rescaled, y_pred_rescaled)
print('R-squared score of the train set:', round(r2_squared,4))

# mean squared error
mse = mean_squared_error(y_train_rescaled, y_pred_rescaled)
rmse = np.sqrt(mse)
print('Root mean squared error of the train set:', round(rmse,4))

# AIC
# trainable_weights includes neuron weights and biases
param_num = model.count_params()
print("Total number of parameters: ", param_num)
no_data_points = xtrain_for_parameter_fitting.shape[0]
aic = no_data_points * np.log(mse) + 2*param_num
print("Akaike Information Criterion:", round(aic, 4))

# Corrected AIC
aic_corrected = aic + \
                2*param_num*(param_num+1)/(no_data_points - param_num -1)
print("Corrected Akaike Information Criterion:", round(aic_corrected, 4))

In [ ]:
# Use the model to calculate predictions for the test set
y_pred = model.predict(xtest)

y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = ['y'] 
y_pred_df.head()

In [ ]:
# Scale back the predictions and original values for the test set
y_pred_rescaled = scale_back(pd.DataFrame(y_pred), ytest_min, ytest_range)
y_pred_rescaled.columns =['y'] 
y_test_rescaled = scale_back(pd.DataFrame(ytest), ytest_min, ytest_range)
y_test_rescaled.head()


In [ ]:
# r-squared with test set
r2_squared = r2_score(y_test_rescaled, y_pred_rescaled)
print('R-squared score of the test set:', round(r2_squared,4))

# mean squared error with the test set
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
rmse = np.sqrt(mse)
print('Root mean squared error of the test set:', round(rmse,4))
 

In [ ]:
# Generate a scatter plot of predicted vs actual data
plt.figure(figsize=(5,5))
plt.scatter(x = y_test_rescaled, y = y_pred_rescaled)
plt.xlabel("'actual'")
plt.ylabel("'predicted'")
plt.title("Scatter plot of predicted vs actual data")

In [ ]:
# Add a time column for plotting purposes
y_test_rescaled['time'] = y_test_rescaled.index
y_test_rescaled = y_test_rescaled.sort_values(by=['time'])
y_actual = pd.DataFrame(y_test_rescaled['y'])
y_actual.reset_index(drop=True, inplace=True)

y_pred_rescaled['time'] = y_test_rescaled.index
y_pred_rescaled = y_pred_rescaled.sort_values(by=['time'])
y_hat = pd.DataFrame(y_pred_rescaled['y'])
y_hat.reset_index(drop=True, inplace=True)

time = y_test_rescaled['time']

# To export the predictions and test values to a CSV file
df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv', index = False)
y_test_rescaled.to_csv(r'ytest.csv', index = False)
y_pred_rescaled.to_csv(r'yhat.csv', index = False)


In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(time, y_actual, linestyle='solid', color='r')
plt.plot(time, y_hat, linestyle='dashed', color='b')

plt.legend(['Actual','Predicted'], loc='best', prop={'size': 14})
plt.title('Actual vs Predicted values', weight='bold', fontsize=16)
#plt.ylabel('demand', weight='bold', fontsize=14)
#plt.xlabel('time', weight='bold', fontsize=14)
plt.xticks(weight='bold', fontsize=12, rotation=45)
plt.yticks(weight='bold', fontsize=12)
plt.grid(color = 'y', linewidth='0.5')
plt.show()

